# Exercise 1

In this exercise you will be implementing a simple aligner. You will use to to align the following read: 'ACATACACATGTCCTGTTTTGATGTCCTATAATTAATTTTCTCTCCGTTTTTAACTTTTATCTATCTTATTAATGT'
to the refernece sequence located in the file example_human_reference.fasta (contig 20)

## Seed phase

* Implement a simple hash-based aligner in Python
    * A dict can be used to create the index
    * Create an index for each kmer in a sequence
    * Create this index for the example fasta file (in the data directory, used in Week 5)



* To map a read, find locations for each kmer in the read
    * Mind the offset from the beginning of the read
    * Find the region with most kmers mapping to it
    * collections.Counter() may be useful (pass a list to Counter(), and it will return the number of times each element occurs in the list)

In [36]:
import pysam # pip install pysam
import sys
from collections import Counter
from itertools import chain

Reading necessary data: reference sequence chr 20, read sequence, kmer size

In [37]:
fasta = pysam.FastaFile("/sbgenomics/project-files/example_human_reference.fasta").fetch('20')
read = 'ACATACACATGTCCTGTTTTGATGTCCTATAATTAATTTTCTCTCCGTTTTTAACTTTTATCTATCTTATTAATGT'
k = 10

STEP 1: Create index for provided fasta file

In [38]:
# function to create index out of reference
def create_index(fasta, k):
    kmers = {}
    for i, x in enumerate(fasta[:-k]):
        kmer = fasta[i:i+k]
        if kmer not in kmers:
            kmers[kmer] = []
        kmers[kmer].append(i)
    return kmers

STEP 2: Get number of unique k-mers and number kmers with of colisions

In [39]:
index = create_index(fasta, k)
print("Number of unique k-mers:", len(index))
print("Number of k-mers with collisions:", len({k:v for k,v in (index).items() if len(v)>1}))

Number of unique k-mers: 9759
Number of k-mers with collisions: 213


STEP 3: Create seed function which will return dict with reference positions as keys and with number of supporting kmers as values

In [40]:
def seed_read1(index, k, read):
    return Counter(chain.from_iterable([[y-i for y in index.get(read[i:i+k], [])] for i in range(len(read) - k +1)]))

In [41]:
def seed_read2(index, k, read):
    result = []
    for i,c in enumerate(read):
        if i+k-1 == len(read):
            break
        kmer_results = []
        for kmer_position in index[read[i:i+k]]:
            #we return starting position of a read on the reference, not kmer on the reference, so we need to subtrac i
            kmer_results.append(kmer_position-i)
        result.append(kmer_results)
    return Counter(chain.from_iterable(result))

# DODATO NAKON PREDAVANJA:
Izlistamo sve k-mere ako zelimo da vidimo koliko ih ima i koji su da bismo bolje razumeli seed korak. 

In [58]:
def kmerize(read,k):
  read_len=len(read)
#k-mer should not be longer than read;  
  if k>=read_len:
    print ("The specified k-mer size is greater or equal to the length of the sequence. Please specify another value.")
    return 
  step_size=k-1
  i=0
  kmers=[]
  while i+step_size<read_len:
    if read[i:i+k] not in kmers:
        kmers.append(read[i:i+k])
    i+=1
  return (kmers)

In [59]:
ks = kmerize(read, 10)

In [61]:
print(ks)

['ACATACACAT', 'CATACACATG', 'ATACACATGT', 'TACACATGTC', 'ACACATGTCC', 'CACATGTCCT', 'ACATGTCCTG', 'CATGTCCTGT', 'ATGTCCTGTT', 'TGTCCTGTTT', 'GTCCTGTTTT', 'TCCTGTTTTG', 'CCTGTTTTGA', 'CTGTTTTGAT', 'TGTTTTGATG', 'GTTTTGATGT', 'TTTTGATGTC', 'TTTGATGTCC', 'TTGATGTCCT', 'TGATGTCCTA', 'GATGTCCTAT', 'ATGTCCTATA', 'TGTCCTATAA', 'GTCCTATAAT', 'TCCTATAATT', 'CCTATAATTA', 'CTATAATTAA', 'TATAATTAAT', 'ATAATTAATT', 'TAATTAATTT', 'AATTAATTTT', 'ATTAATTTTC', 'TTAATTTTCT', 'TAATTTTCTC', 'AATTTTCTCT', 'ATTTTCTCTC', 'TTTTCTCTCC', 'TTTCTCTCCG', 'TTCTCTCCGT', 'TCTCTCCGTT', 'CTCTCCGTTT', 'TCTCCGTTTT', 'CTCCGTTTTT', 'TCCGTTTTTA', 'CCGTTTTTAA', 'CGTTTTTAAC', 'GTTTTTAACT', 'TTTTTAACTT', 'TTTTAACTTT', 'TTTAACTTTT', 'TTAACTTTTA', 'TAACTTTTAT', 'AACTTTTATC', 'ACTTTTATCT', 'CTTTTATCTA', 'TTTTATCTAT', 'TTTATCTATC', 'TTATCTATCT', 'TATCTATCTT', 'ATCTATCTTA', 'TCTATCTTAT', 'CTATCTTATT', 'TATCTTATTA', 'ATCTTATTAA', 'TCTTATTAAT', 'CTTATTAATG', 'TTATTAATGT']


In [71]:
print(ks[31])

ATTAATTTTC


In [82]:
len(ks)

67

Ovih 67 k-mera koristimo kao seed-ove i gledamo gde mozemo da mapiramo na referenci:

In [47]:
k = 10

In [48]:
seed_read1(index,k,read)

Counter({5793: 67, 792: 2})

In [49]:
seed_read2(index,k,read)

Counter({5793: 67, 792: 2})

Dva k-mera se mapiraju na dve razlicite pozicije kada koristimo zadate vrednosti. Svejedno, pozicija koja ima najvise mapirani seed-ova nam je znak da se read tu najbolje uklapa. Sad cemo to proveriti i videti kako zapravo izgleda referentna sekvenca duzine reada na izvucenim pozicijama:

**5793 i 792 su pozicije na referenci gde bi se READ potencijalno mapirao, ne gde se taj seed, odnosno k-mer nalazi!**

Proverite duzinu read-a i onda izvucite string iz fasta-e gde bi se taj read mapirao. Da li se nas read bolje uklapa u string na referenci koji pocinje na poziciji 792 ili 5793?

In [83]:
len(read)

76

In [84]:
print(read)

ACATACACATGTCCTGTTTTGATGTCCTATAATTAATTTTCTCTCCGTTTTTAACTTTTATCTATCTTATTAATGT


In [85]:
fasta[792:(792 + 76)]

'CTAATCGCCTCTAGCTGAAATTTTAATTTTGATTAATTTTCTTTTTCTCCATTGGTTTCGTGTGTGTGTGGAGGTA'

In [86]:
fasta[5793:(5793 + 76)]

'ACATACACATGTCCTGTTTTGATGTCCTATAATTAATTTTCTCTCCGTTTTTAACTTTTATCTATCTTATTAATGT'

Da li je sad malo jasnije? :) 

STEP 4: Possible improvements? Filter out all kmers that have more than n mapping positions

In [87]:
def seed_read2_with_improvements(index, k, read, n=2):
    result = []
    for i,c in enumerate(read):
        if i+k-1 == len(read):
            break
        kmer_results = []
        for kmer_position in index[read[i:i+k]]:
            #we return starting position of a read on the reference, not kmer on the reference, so we need to subtrac i
            kmer_results.append(kmer_position-i)
        if len(kmer_results)<=n:
            result.append(kmer_results)
    return Counter(chain.from_iterable(result))

In [88]:
seed_read2_with_improvements(index,k,read)

Counter({5793: 67, 792: 2})

In [89]:
seed_read2_with_improvements(index,k,read, 1)

Counter({5793: 65})